In [1]:
import yfinance as yf  
import numpy as np

tickers = ['TSLA','FB','AAPL','AMZN','MSFT'] # list of tickers for stocks in my portfolio
 
daily19_20 = yf.download(tickers,'2019-01-01','2020-01-01')['Adj Close']# dataframe of adjusted close prices for 
                                                                        # each stock from 2018.12.31 to 2019.12.31
                                                                        # from yafoo finance
                                                                        # (2019.01.01 - not a trading day)
prices31_12_20 = yf.download(tickers,'2021-01-01','2021-01-01')['Close']  # dataframe of closing prices for 
                                                                            # each stock on 31.12.2020
display(daily19_20, prices31_12_20) 

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed


,AAPL,AMZN,FB,MSFT,TSLA
Date,,,,,
2018-12-31,38.461140,1501.969971,131.089996,98.813278,66.559998
2019-01-02,38.505024,1539.130005,135.679993,98.375481,62.023998
2019-01-03,34.669640,1500.280029,131.740005,94.756454,60.071999
2019-01-04,36.149662,1575.390015,137.949997,99.163513,63.537998
2019-01-07,36.069202,1629.510010,138.050003,99.289970,66.991997
...,...,...,...,...,...
2019-12-24,70.353882,1789.209961,205.119995,155.372406,85.050003
2019-12-26,71.749733,1868.770020,207.789993,156.645950,86.188004
2019-12-27,71.722488,1869.800049,208.100006,156.932251,86.075996


,AAPL,AMZN,FB,MSFT,TSLA
Date,,,,,
2020-12-31,132.690002,3256.929932,273.160004,222.419998,705.669983


In [2]:
daily_returns = (daily19_20 - daily19_20.shift(1)) / daily19_20.shift(1)#using formula for rate of return from moodle, 
                                                                        # r=(X1-X0)/X0
daily_returns                                                           # alternatively, dr = daily19_20.pct_change()

,AAPL,AMZN,FB,MSFT,TSLA
Date,,,,,
2018-12-31,NaN,NaN,NaN,NaN,NaN
2019-01-02,0.001141,0.024741,0.035014,-0.004431,-0.068149
2019-01-03,-0.099607,-0.025242,-0.029039,-0.036788,-0.031472
2019-01-04,0.042689,0.050064,0.047138,0.046509,0.057697
2019-01-07,-0.002226,0.034353,0.000725,0.001275,0.054361
...,...,...,...,...,...
2019-12-24,0.000951,-0.002114,-0.005141,-0.000191,0.014384
2019-12-26,0.019840,0.044467,0.013017,0.008197,0.013380
2019-12-27,-0.000380,0.000551,0.001492,0.001828,-0.001300


In [3]:
# mean returns vector code:
r = daily_returns.iloc[1:] # delete first row with NaNs
n = len(r)
m = len(r.columns)
r_mean = []

for i in range(m):
    terms = 0
    for k in range(n):
        terms += r.iat[k,i]
    r_mean.append(terms / n)

r_mean = np.array(r_mean)

# we could use:
r_mean2 = np.array(r.mean(axis=0))
r_mean, r_mean2
# which gives same results

(array([0.0026646 , 0.00092645, 0.00193313, 0.00188339, 0.0013907 ]),
 array([0.0026646 , 0.00092645, 0.00193313, 0.00188339, 0.0013907 ]))

In [4]:
Sigma = []
for i in range(m):
    for j in range(m):
        terms = 0
        for k in range(n):
            terms += (r.iat[k,i] - r_mean[i]) * (r.iat[k,j] - r_mean[j])
        cov = terms / (n) # biased estimator
        Sigma.append(cov)
Sigma = np.reshape(Sigma,(m,m))

# we could use:
Sigma2 = (r.cov())

display(Sigma, Sigma2)
# Results are slighly different because command .cov uses unbiased estimator (n-1)

#Annualised results
r_mean = r_mean * 252.75
Sigma = Sigma * 252.75

display(r_mean, Sigma)

array([[0.00027004, 0.00013941, 0.00013585, 0.00012737, 0.00016825],
       [0.00013941, 0.00020745, 0.00015685, 0.00012846, 0.000123  ],
       [0.00013585, 0.00015685, 0.00030717, 0.0001144 , 0.00011855],
       [0.00012737, 0.00012846, 0.0001144 , 0.0001554 , 0.00011074],
       [0.00016825, 0.000123  , 0.00011855, 0.00011074, 0.00096143]])

,AAPL,AMZN,FB,MSFT,TSLA
AAPL,0.000271,0.000140,0.000136,0.000128,0.000169
AMZN,0.000140,0.000208,0.000157,0.000129,0.000123
FB,0.000136,0.000157,0.000308,0.000115,0.000119
MSFT,0.000128,0.000129,0.000115,0.000156,0.000111
TSLA,0.000169,0.000123,0.000119,0.000111,0.000965


array([0.67347669, 0.23416066, 0.48859785, 0.47602737, 0.35150046])

array([[0.0682527 , 0.03523713, 0.03433688, 0.03219264, 0.04252531],
       [0.03523713, 0.05243306, 0.03964468, 0.03246856, 0.03108779],
       [0.03433688, 0.03964468, 0.07763619, 0.02891381, 0.02996463],
       [0.03219264, 0.03246856, 0.02891381, 0.03927752, 0.02799044],
       [0.04252531, 0.03108779, 0.02996463, 0.02799044, 0.24300126]])

In [5]:
from scipy.optimize import minimize

w = np.random.random(len(r_mean))# generates random number of n-number of weights to start optimisation
ones = np.ones(len(r_mean))      # generates vector of ones
r_target = [0.05,0.1,0.2]   # list of chosen expected returns
w_target = []               
var = []

def objective(w):
    return np.dot(w, np.dot(Sigma, w))# w is 1D vector, S is symmetric, no need to transpose in python 
def constraint1(w):
    return np.dot(w,ones)-1
for rt in r_target:
    def constraint2(w):
        return np.dot(w,r_mean) - rt
    cons = ({'type' : 'eq' , 'fun' : constraint1}, 
            {'type' : 'eq' , 'fun' : constraint2}) 
    solution = minimize(fun = objective, x0 = w, method = 'SLSQP',constraints = cons)
    w = solution.x
    w_target.append(w)
    var.append(objective(w))

w_target = np.array(w_target).round(2)
var = np.array(var).round(6)
prices = np.array(daily19_20.values[-1])
shares = np.divide(w_target, prices)*1000000
shares = shares.round(1)

r_target, w_target, var, shares

([0.05, 0.1, 0.2],
 array([[-0.53,  1.27, -0.13,  0.29,  0.1 ],
        [-0.45,  1.14, -0.09,  0.32,  0.09],
        [-0.3 ,  0.85, -0.04,  0.42,  0.07]]),
 array([0.071085, 0.063053, 0.050128]),
 array([[-7292.7,   687.3,  -633.4,  1862.7,  1195.2],
        [-6191.9,   616.9,  -438.5,  2055.4,  1075.7],
        [-4127.9,   460. ,  -194.9,  2697.7,   836.7]]))

In [6]:
value = np.dot(shares, prices31_12_20.T).round(2)
value

array([[2355518.51],
       [2284067.45],
       [2087674.33]])

In [7]:
np.sqrt(var)

array([0.2666177 , 0.25110356, 0.22389283])